In [1]:
import pandas as pd
import numpy as np
import itertools
from functools import reduce
import os
import multiprocessing
import time
import warnings
from tqdm import tqdm
import datetime

warnings.filterwarnings("ignore")

pd.options.display.width = 1200
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 10
pd.options.mode.chained_assignment = None

In [2]:
def priceSensitivityOrderFlowImbalance(df):  
    deltaP = 10000*df["LastPrice"].diff().fillna(0)
    bidQty1 = df["BidOrderQty1"].fillna(0)
    askQty1 = df["OfferOrderQty1"].fillna(0)
    NVOL = bidQty1 - askQty1

    # 线性回归公式 linearRegression(y, x): (len(x)*sum(x*y) - sum(x)*sum(y)) / (len(x)*sum(x*x) - sum(x)*sum(x))
    res = (len(NVOL)*sum(NVOL*deltaP) - sum(NVOL)*sum(deltaP)) / (len(NVOL)*sum(NVOL*NVOL)- sum(NVOL)*sum(NVOL))
    return res

In [ ]:
df = pd.read_csv("/ssd/ssd3/data/oneStock_oneFile_TL/20230201/snapshot/000001.csv")
t0 = time.time()
res = priceSensitivityOrderFlowImbalance(df)
print("cal time: ", time.time() - t0, "s")
print(res)

In [4]:
def pool_func(tick_obj, trade_path_obj):
    single_tick_res = pd.DataFrame(columns=["DATE","priceSensitivityOrderFlowImbalance"])
    tmp_date = trade_path_obj.split('/')[-2]
    # print(tmp_date)
    tmp_date = tmp_date[0:4] + "-" + tmp_date[4:6] + "-" + tmp_date[6:8]
    # print(tmp_date)
    for tick in tqdm(tick_obj):
        single_tick_res.at[tick[:6], "DATE"] = tmp_date
        try:
            df = pd.read_csv(os.path.join(trade_path_obj, tick))

            Indicator = priceSensitivityOrderFlowImbalance(df)
            # print(Indicator)
            # print("开盘后大单净买入占比:", Indicator)
            single_tick_res.at[tick[:6], "priceSensitivityOrderFlowImbalance"] = Indicator

        except Exception as error:
            single_tick_res.at[tick[:6], "priceSensitivityOrderFlowImbalance"] = np.nan
            continue

    return single_tick_res


class multi_task_split:

    def __init__(self, data, processes_to_use):
        self.data = data
        self.processes_to_use = processes_to_use

    def num_of_jobs(self):
        return min(len(self.data), self.processes_to_use, multiprocessing.cpu_count())

    def split_args(self):
        q, r = divmod(len(self.data), self.num_of_jobs())
        return (self.data[i * q + min(i, r): (i + 1) * q + min(i + 1, r)] for i in range(self.num_of_jobs()))

In [ ]:
n_use = 24
# 路径修改为存放数据路径
trade_path = r"/ssd/ssd3/data/oneStock_oneFile_TL/20230201/snapshot"
stock_pool = os.listdir(trade_path)
processes_decided = multi_task_split(stock_pool, n_use).num_of_jobs()
print("进程数：", processes_decided)
split_args_to_process = list(multi_task_split(stock_pool, n_use).split_args())
args = [(split_args_to_process[i], trade_path) for i in range(len(split_args_to_process))]
print("#" * 50 + "Multiprocessing Start" + "#" * 50)
t0 = time.time()
with multiprocessing.Pool(processes=processes_decided) as pool:
    res = tqdm(pool.starmap(pool_func, args))
    print("cal time: ", time.time() - t0, "s")
    res_combined = pd.concat(res, axis=0)
    pool.close()
    print("cal time: ", time.time() - t0, "s")
print(res_combined)